In [1]:
import config
from model import Translator
from transformers import AutoTokenizer

src_tokenizer = AutoTokenizer.from_pretrained(config.SRC_MODEL_NAME)
tgt_tokenizer = AutoTokenizer.from_pretrained(config.TGT_MODEL_NAME)

translator = Translator.load_from_checkpoint(
    "../checkpoints/last.ckpt",
    src_tokenizer=src_tokenizer,
    tgt_tokenizer=tgt_tokenizer,
)

/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/hoang/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls

In [2]:
translator.greedy_translate("When i was little")

'Khi nhỏ bé nhỏ.'

In [3]:
translator.beam_translate("When I was little, I thought my country was the best in the world.")

[(0.28848347067832947,
  'Khi nhỏ, tôi nghĩ đất nước mình là tốt nhất thế giới.'),
 (0.34126970171928406,
  'Khi nhỏ, tôi nghĩ rằng đất nước tôi là tốt nhất thế giới.'),
 (0.6581437587738037,
  'Khi nhỏ, tôi nghĩ rằng đất nước mình là tốt nhất thế giới.')]

In [11]:
import torch
ckpt = torch.load("../checkpoints/last-v1.ckpt", map_location="cpu")
ckpt.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters'])

In [12]:
ckpt["epoch"]

1

In [4]:
from datasets import load_dataset

iwslt = load_dataset("mt_eng_vietnamese", "iwslt2015-vi-en", split="test")

Found cached dataset mt_eng_vietnamese (/home/hoang/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-vi-en/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)


In [5]:
iwslt[0]["translation"]

{'en': 'When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;',
 'vi': 'Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;'}

In [7]:
translator.beam_translate(iwslt[2]["translation"]["en"], beam_size=3)

[(0.2620276212692261,
  'Chúng tôi dành rất nhiều thời gian nghiên cứu về lịch sử của Joel, nhưng chúng tôi chưa biết được nhiều về thế giới bên ngoài, ngoại trừ nước Mỹ, Nhật Bản là kẻ thù.'),
 (0.3884652853012085,
  'Chúng tôi dành rất nhiều thời gian nghiên cứu về lịch sử của Jopest, nhưng chúng tôi chưa biết được nhiều về thế giới bên ngoài, ngoại trừ nước Mỹ, Nhật Bản là kẻ thù.'),
 (0.5975554585456848,
  'Chúng tôi dành rất nhiều thời gian nghiên cứu về lịch sử của Joel, nhưng chúng tôi chưa từng học được nhiều về thế giới bên ngoài, ngoại trừ nước Mỹ, Nhật Bản là kẻ thù.')]

In [ ]:
iwslt[2]["translation"]["vi"]

'Ở trường , chúng tôi dành rất nhiều thời gian để học về cuộc đời của chủ tịch Kim II- Sung , nhưng lại không học nhiều về thế giới bên ngoài , ngoại trừ việc Hoa Kỳ , Hàn Quốc và Nhật Bản là kẻ thù của chúng tôi .'

In [ ]:
translator.greedy_translate(iwslt[2]["translation"]["en"])

'Ở, chúng tôi đã dành rất nhiều thời gian nghiên cứu về lịch sử của Joel, ngoại trừ nước ngoài, ngoại trừ nước Mỹ, ngoại trừ nước Mỹ, Nhật Bản là kẻ thù địch.'